**基本介绍**

随机场：当给每一个位置中按照某种分布随机赋予相空间的一个值之后，其全体就叫做随机场。

马尔科夫随机场是随机场的特例，它假设随机场中某一个位置的赋值仅仅与和它相邻的位置的赋值有关，和与其不相邻的位置的赋值无关。

条件随机场(Conditional Random Field，CRF)又是马尔科夫随机场的特例，是给定随机变量$X$条件下，随机变量$Y$的马尔科夫场。

线性链条件随机场中的条件概率模型$P(Y|X)$，$Y$是输出变量，表示标记序列；而$X$是输入变量，表示需要标注的观测序列。设$X=(X_1, X_2,...,X_n)$和$Y=(Y_1, Y_2,...,Y_n)$均为线性链表示的随机变量序列，条件随机场

$$P(Y_i|X, Y_1,Y_2,...Y_{i-1}, Y_{i+1},...,Y_n)=P(Y_i|X,Y_{i-1}, Y_{i+1})$$


在CRF中，为了对其进行参数表达，设计了两类特征函数。

1、节点特征函数，只跟当前节点有关

$$s_l(y_i, x_i, i), l = 1,2,3,...,L$$

2、局部特征函数，跟当前节点和上一个节点有关

$$t_k(y_{i-1}, y_i, x_i, i), k = 1,2,3,...,K$$

无论是节点特征函数还是局部特征函数，都只能取值0/1。

同时，我们可以为每个特征函数赋予一个权值，用以表达我们对这个特征函数的信任度$\lambda_k$和$\mu_l$。


因此，参数化的CRF可以表示为:
$$P(y|x)=\frac{1}{Z(x)} exp(\sum_{i,k}\lambda_kt_k(y_{i-1}, y_i, x_i, i) + \sum_{i, l}\mu_ls_l(y_i, x_i, i))$$

其中，$Z(x)$为规范化因子，$Z(x)=\sum_yexp(\sum_{i,k}\lambda_kt_k(y_{i-1}, y_i, x_i, i) + \sum_{i, l}\mu_ls_l(y_i, x_i, i))$

特别地，将两个函数统一成$$f_k(y_{i-1}, y_i, x_i, i)=\begin{cases}
t_k(y_{i-1}, y_i, x_i, i) & k=1,2,3,...,K\\
s_k(y_i, x_i, i)& k=K+l, l=1,2,3,...,L
\end{cases}$$
以及$$\omega_k=\begin{cases}
\lambda_k & k=1,2,3,...,K\\
\mu_k& k=K+l, l=1,2,3,...,L
\end{cases}$$

所以原式就可以简化成：

$$P(y|x)=\frac{1}{Z(x)} exp(\sum_{k=1}^{K+L}\omega_k f_k(y_{i-1}, y_i, x_i, i))$$

这样，$\omega$就成了要训练的参数。

**CRF解决了MEMM模型的标注偏差问题，打破了齐次markov假设，使得模型更加合理**

参考：https://zhuanlan.zhihu.com/p/29989121

-------------------
**要解决的问题**

线性CRF需要解决三个核心问题，包括基于前向-后向的概率估计算法、基于极大似然和牛顿优化的学习算法以及基于维特比算法的预测算法。


文本序列中的每一个汉字都应当这样分配一套参数权重矩阵,若把每一个汉字的所有特征列个七七八八，实际上也就对汉语中所有的汉字和其词汇做了一个特征大汇总，那这就是一本大型的汉语字典。放在模型中，这样的特征数量往往在百万级起步.

如果说，采用LSTM 模型来加入CRF模型上，叫做Bi-LSTM-CRF 模型，那么最原始的经典CRF模型，其实可以被称作 Logistics-CRF，也即多类别逻辑回归CRF模型。

总结一下，转移参数起到的作用，即限制某些不可能达成的标签转移。否则作用微乎其微.

**参考：http://www.jionlp.com/lecture/logistics_crf**

In [4]:
import re
import os

class CorpusProcess(object):

    def __init__(self):
        """初始化"""
        self.train_corpus_path ="./199801.txt"
        self.process_corpus_path ="result-rmrb.txt"
        self._maps = {u't': u'T', u'nr': u'PER', u'ns': u'ORG', u'nt': u'LOC'}

    def read_corpus_from_file(self, file_path):
        """读取语料"""
        f = open(file_path, 'r', encoding='utf-8')
        lines = f.readlines()
        f.close()
        return lines

    def write_corpus_to_file(self, data, file_path):
        """写语料"""
        f = open(file_path, 'wb')
        f.write(data)
        f.close()

    def q_to_b(self, q_str):
        """全角转半角"""
        b_str = ""
        for uchar in q_str:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif 65374 >= inside_code >= 65281:  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            b_str += chr(inside_code)
        return b_str

    def b_to_q(self, b_str):
        """半角转全角"""
        q_str = ""
        for uchar in b_str:
            inside_code = ord(uchar)
            if inside_code == 32:  # 半角空格直接转化
                inside_code = 12288
            elif 126 >= inside_code >= 32:  # 半角字符（除空格）根据关系转化
                inside_code += 65248
            q_str += chr(inside_code)
        return q_str

    def pre_process(self):
        """语料预处理 """
        if os.path.exists(self.process_corpus_path):
            return
        lines = self.read_corpus_from_file(self.train_corpus_path)
        new_lines = []
        for line in lines:
            words = self.q_to_b(line.strip()).split(u'  ')
            pro_words = self.process_t(words)
            pro_words = self.process_nr(pro_words)
            pro_words = self.process_k(pro_words)
            new_lines.append('  '.join(pro_words[1:]))
        self.write_corpus_to_file(data='\n'.join(new_lines).encode('utf-8'), file_path=self.process_corpus_path)

    def process_k(self, words):
        """处理大粒度分词,合并语料库中括号中的大粒度分词,类似：[国家/n  环保局/n]nt """
        pro_words = []
        index = 0
        temp = u''
        while True:
            word = words[index] if index < len(words) else u''
            if u'[' in word:
                temp += re.sub(pattern=u'/[a-zA-Z]*', repl=u'', string=word.replace(u'[', u''))
            elif u']' in word:
                w = word.split(u']')
                temp += re.sub(pattern=u'/[a-zA-Z]*', repl=u'', string=w[0])
                pro_words.append(temp + u'/' + w[1])
                temp = u''
            elif temp:
                temp += re.sub(pattern=u'/[a-zA-Z]*', repl=u'', string=word)
            elif word:
                pro_words.append(word)
            else:
                break
            index += 1
        return pro_words

    def process_nr(self, words):
        """ 处理姓名，合并语料库分开标注的姓和名，类似：温/nr  家宝/nr"""
        pro_words = []
        index = 0
        while True:
            word = words[index] if index < len(words) else u''
            if u'/nr' in word:
                next_index = index + 1
                if next_index < len(words) and u'/nr' in words[next_index]:
                    pro_words.append(word.replace(u'/nr', u'') + words[next_index])
                    index = next_index
                else:
                    pro_words.append(word)
            elif word:
                pro_words.append(word)
            else:
                break
            index += 1
        return pro_words

    def process_t(self, words):
        """处理时间,合并语料库分开标注的时间词，类似： （/w  一九九七年/t  十二月/t  三十一日/t  ）/w   """
        pro_words = []
        index = 0
        temp = u''
        while True:
            word = words[index] if index < len(words) else u''
            if u'/t' in word:
                temp = temp.replace(u'/t', u'') + word
            elif temp:
                pro_words.append(temp)
                pro_words.append(word)
                temp = u''
            elif word:
                pro_words.append(word)
            else:
                break
            index += 1
        return pro_words

    def pos_to_tag(self, p):
        """由词性提取标签"""
        t = self._maps.get(p, None)
        return t if t else u'O'

    def tag_perform(self, tag, index):
        """标签使用BIO模式"""
        if index == 0 and tag != u'O':
            return u'B_{}'.format(tag)
        elif tag != u'O':
            return u'I_{}'.format(tag)
        else:
            return tag

    def pos_perform(self, pos):
        """去除词性携带的标签先验知识"""
        if pos in self._maps.keys() and pos != u't':
            return u'n'
        else:
            return pos

    def initialize(self):
        """初始化 """
        lines = self.read_corpus_from_file(self.process_corpus_path)
        words_list = [line.strip().split('  ') for line in lines if line.strip()]
        del lines
        self.init_sequence(words_list)

    def init_sequence(self, words_list):
        """初始化字序列、词性序列、标记序列 """
        words_seq = [[word.split(u'/')[0] for word in words] for words in words_list]
        pos_seq = [[word.split(u'/')[1] for word in words] for words in words_list]
        tag_seq = [[self.pos_to_tag(p) for p in pos] for pos in pos_seq]
        self.pos_seq = [[[pos_seq[index][i] for _ in range(len(words_seq[index][i]))]
                         for i in range(len(pos_seq[index]))] for index in range(len(pos_seq))]
        self.tag_seq = [[[self.tag_perform(tag_seq[index][i], w) for w in range(len(words_seq[index][i]))]
                         for i in range(len(tag_seq[index]))] for index in range(len(tag_seq))]
        self.pos_seq = [[u'un'] + [self.pos_perform(p) for pos in pos_seq for p in pos] + [u'un'] for pos_seq in
                        self.pos_seq]
        self.tag_seq = [[t for tag in tag_seq for t in tag] for tag_seq in self.tag_seq]
        self.word_seq = [[u'<BOS>'] + [w for word in word_seq for w in word] + [u'<EOS>'] for word_seq in words_seq]

    def extract_feature(self, word_grams):
        """特征选取"""
        features, feature_list = [], []
        for index in range(len(word_grams)):
            for i in range(len(word_grams[index])):
                word_gram = word_grams[index][i]
                feature = {u'w-1': word_gram[0], u'w': word_gram[1], u'w+1': word_gram[2],
                           u'w-1:w': word_gram[0] + word_gram[1], u'w:w+1': word_gram[1] + word_gram[2],
                           # u'p-1': self.pos_seq[index][i], u'p': self.pos_seq[index][i+1],
                           # u'p+1': self.pos_seq[index][i+2],
                           # u'p-1:p': self.pos_seq[index][i]+self.pos_seq[index][i+1],
                           # u'p:p+1': self.pos_seq[index][i+1]+self.pos_seq[index][i+2],
                           u'bias': 1.0}
                feature_list.append(feature)
            features.append(feature_list)
            feature_list = []
        return features

    def segment_by_window(self, words_list=None, window=3):
        """窗口切分"""
        words = []
        begin, end = 0, window
        for _ in range(1, len(words_list)):
            if end > len(words_list): break
            words.append(words_list[begin:end])
            begin = begin + 1
            end = end + 1
        return words

    def generator(self):
        """训练数据"""
        word_grams = [self.segment_by_window(word_list) for word_list in self.word_seq]
        features = self.extract_feature(word_grams)
        return features, self.tag_seq

In [5]:
import joblib
import sklearn_crfsuite
from sklearn_crfsuite import metrics


class CRF_NER(object):

    def __init__(self):
        """初始化参数"""
        self.algorithm = "lbfgs"
        self.c1 = "0.1"
        self.c2 = "0.1"
        self.max_iterations = 100
        self.model_path ="model.pkl"
        self.corpus = CorpusProcess()  # Corpus 实例
        self.corpus.pre_process()  # 语料预处理
        self.corpus.initialize()  # 初始化语料
        self.model = None

    def initialize_model(self):
        """初始化"""
        algorithm = self.algorithm
        c1 = float(self.c1)
        c2 = float(self.c2)
        max_iterations = int(self.max_iterations)
        self.model = sklearn_crfsuite.CRF(algorithm=algorithm, c1=c1, c2=c2,
                                          max_iterations=max_iterations, all_possible_transitions=True)

    def train(self):
        """训练"""
        self.initialize_model()
        x, y = self.corpus.generator()
        x_train, y_train = x[500:], y[500:]
        x_test, y_test = x[:500], y[:500]
        self.model.fit(x_train, y_train)
        labels = list(self.model.classes_)
        labels.remove('O')
        y_predict = self.model.predict(x_test)
        metrics.flat_f1_score(y_test, y_predict, average='weighted', labels=labels)
        sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))
        print(metrics.flat_classification_report(y_test, y_predict, labels=sorted_labels, digits=3))
        self.save_model()

    def predict(self, sentence):
        """预测"""
        self.load_model()
        
        u_sent = self.corpus.q_to_b(sentence)
        word_lists = [[u'<BOS>'] + [c for c in u_sent] + [u'<EOS>']]
        word_grams = [self.corpus.segment_by_window(word_list) for word_list in word_lists]
        features = self.corpus.extract_feature(word_grams)
        y_predict = self.model.predict(features)
        entity = u''
        for index in range(len(y_predict[0])):
            if y_predict[0][index] != u'O':
                if index > 0 and y_predict[0][index][-1] != y_predict[0][index - 1][-1]:
                    entity += u' '
                entity += u_sent[index]
            elif entity[-1] != u' ':
                entity += u' '
        return entity

    def load_model(self):
        """加载模型 """
        self.model = joblib.load(self.model_path)

    def save_model(self):
        """保存模型"""
        joblib.dump(self.model, self.model_path)

In [ ]:
ner = CRF_NER()
#训练模型，当训练完毕后，就可以直接加载模型参数，不用再次训练了
mode = ner.train()

result1 = ner.predict(u'新华社北京十二月三十一日电(中央人民广播电台记者刘振英、新华社记者张宿堂)今天是一九九七年的最后一天。')
print(result1)
result2=ner.predict(u'北京和天津。')
print(result2)